# ZnTrack Parameters with dataclasses

To structure the parameters used in a Node it can be useful to pass them as a dataclass. The following Notebook will illustrate a small Example.

In [1]:
import pathlib

from zntrack import config
import znjson
import dataclasses

config.nb_name = "dataclasses_for_parameters.ipynb"

In [2]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [3]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmpv_5nwynt/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
from zntrack import Node, zn, Project
import random

In [5]:
@dataclasses.dataclass
class Parameter:
    start: int
    stop: int
    step: int = 1


class ParameterConverter(znjson.ConverterBase):
    instance = Parameter
    representation = "Parameter"

    def encode(self, obj) -> str:
        return dataclasses.asdict(obj)

    def decode(self, value: str):
        return Parameter(**value)


class ComputeRandomNumber(Node):
    param: Parameter = zn.params()
    number = zn.outs()

    # register the Converter
    # ----------------------
    # you can register the converter anywhere,
    # as long as it is executed before
    # the first time the node is instantiated
    _ = znjson.config.register(ParameterConverter)

    def __init__(self, param: Parameter = None, **kwargs):
        super().__init__(**kwargs)
        self.param = param

    def run(self):
        self.number = random.randrange(self.param.start, self.param.stop, self.param.step)

In [6]:
parameter = Parameter(start=100, stop=200)
with Project() as proj:
    ComputeRandomNumber(param=parameter)
proj.run(repro=True)

Running DVC command: 'stage add --name ComputeRandomNumber --force ...'


Creating 'dvc.yaml'
Adding stage 'ComputeRandomNumber' in 'dvc.yaml'

To track the changes with git, run:

	git add nodes/ComputeRandomNumber/.gitignore dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true


Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
[NbConvertApp] Converting notebook dataclasses_for_parameters.ipynb to script
[NbConvertApp] Writing 1907 bytes to dataclasses_for_parameters.py
Running DVC command: 'repro'


Running stage 'ComputeRandomNumber':
> zntrack run src.ComputeRandomNumber.ComputeRandomNumber --name ComputeRandomNumber
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


In [7]:
print(ComputeRandomNumber.from_rev().number)
print(ComputeRandomNumber.from_rev().param)

134
Parameter(start=100, stop=200, step=1)


The arguments of the dataclass are saved in the `params.yaml` file and can also be modified there.

In [8]:
print(pathlib.Path("params.yaml").read_text())

ComputeRandomNumber:
    param:
        _type: Parameter
        value:
            start: 100
            step: 1
            stop: 200



In [9]:
temp_dir.cleanup()